In [ ]:
                #转换754函数

import numpy as np
#将科学计数法关闭（suppress=True）
#打印数组元素的精度设置为15位小数（precision=15）
# 并且不限制数组的打印阈值（threshold=np.inf）
np.set_printoptions(suppress = True, threshold = np.inf, precision = 15)
import bitstring, random 
def ieee754H(flt):
    b = bitstring.BitArray(float=flt, length=32)
    return b
def ieee754B(flt):#将浮点数转换为32位的二进制字符串
    aa = bitstring.BitArray(float=flt, length=32)
    aa = str(aa)
    aa = aa[2:]
    aa = bin(int(aa, 16))
    aa = aa[2:]
    aa = aa.rjust(32,'0')
    return aa

In [ ]:
def bitlet_time(arr_w, arr_a):
    mantis = np.empty((24,64)) #24x64的NumPy数组
    exp = np.empty(64)         #64个元素的NumPy数组
    time = 0
    for i in range(64):
        data_w = str(ieee754B(arr_w[i]))#将 arr_w[i] 和 arr_a[i] 转换为二进制字符串
        data_a = str(ieee754B(arr_a[i]))
        e_w = data_w[1:9]   #提取它们的指数部分并将其转换为整数，
        e_a = data_a[1:9]
        exp[i] = int(e_w, 2) + int(e_a, 2)
        
    expMax = exp[0]
    for i in range(64):#找到 exp 数组中的最大值，将其存储在 expMax 变量中
        if(expMax < exp[i]):
            expMax = exp[i]
    
    for i in range(64): 
        #构建字符串 m，这是尾数的二进制表示
        m = "1" + data_w[9:]
        #然后根据 expMax 和浮点数的指数调整尾数的长度和位数
        m = m[:24-(int(expMax-exp[i]))]
        m = m.rjust(24,'0')
        for j in range(24):
            #将调整后的尾数存储在 mantis 数组中的相应位置
            mantis[j][i] = m[j]

    # time = np.empty(5*5*5*5)
    ones = np.empty((24,4))
    for i in range(24):
        for k in range(4):
            one = 0
            for j in range(8):
                if mantis[i][32+8*k+j]:
                    #对 mantis 数组的每个元素进行计算，
                    # 以确定尾数中有多少个二进制位为1，将结果存储在 ones 数组中
                    one = one + 1 
            ones[i][k] = one

    for i in range(24):
        for k in range(4):
            #根据不同的情况，将 ones 数组中的值映射为新的值
            if (ones[i][k] == 0 or ones[i][k] == 1):
                ones[i][k] = k+6
            elif (ones[i][k] == 2):
                ones[i][k] = k+7
            elif (ones[i][k] == (3 or 4)):
                ones[i][k] = k+8
            else:
                ones[i][k] = k+9
    #比较并合并相同的值，以简化 ones 数组
    for i in range(24):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(24):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(24):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1
    #找到 ones 数组中的最大值，将其存储在 time 变量中。
    time = ones[0][0]
    for i in range(8):
        for k in range(4):
            if(time < ones[i][k]):
                time = ones[i][k]

    return time+19

In [ ]:

def Mac_64(arr_w, mac_a64):
    weight_size = int(arr_w.size/1000) #确定 arr_w 数组的大小。
    mac_time = 0

    for i in range(0, weight_size, 64):#从0到 weight_size，每次增加64的步长：
        mac_time = mac_time + bitlet_time(arr_w[i:i+64],mac_a64)#计算bitlet_time时间值
    return mac_time


def Conv_33(arr_w, arr_a, w_shape, a_shape, s):
    #检查输入数组 arr_w 的大小是否不是64的倍数
    if(arr_w.size % 64 != 0):
        #末尾填充零，以确保大小为64的倍数
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0

    for i in range(0 , (a_shape[3] - 2), s):
        #外层for 控制卷积操作的行 
        print("row = ",i)           #row
        for j in range(0 , (a_shape[2] - 2), s):        #channel
            #中层for 控制卷积操作的通道 
            for k in range(a_shape[1]):        #col
                #内层for 控制卷积操作的列 
                
                #遍历一个3x3的窗口
                for c in range(3):
                    for r in range(3):
                        #将窗口内的数据复制到 mac_a64 数组中
                        mac_a64[cnt] =  arr_a[i+c,j+r,k]
                        cnt = cnt + 1#，维护 cnt 计数器
                        if(cnt == 64):
                            #如果 cnt 的值等于64
                            #将 cnt 重置为0，并调用Mac_64 的函数
                            cnt = 0
                            yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "|")

    return yolo_time


def Conv_11(arr_w, arr_a, w_shape, a_shape):
    #确保大小为64的倍数
    if(arr_w.size % 64 != 0):
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0
    
    #控制卷积操作的行
    for i in range(a_shape[3]):
        print("row = ",i)            #row
        #控制卷积操作的通道
        for j in range(a_shape[2]):        #channel
            #控制卷积操作的列
            for k in range(a_shape[1]):        #col
                
                mac_a64[cnt] =  arr_a[i,j,k]
                cnt = cnt + 1
                if(cnt == 64):
                    cnt = 0
                    yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "| ")

    return yolo_time

In [ ]:
import numpy as np
import torch
from PIL import Image #处理图像
from torch.nn import functional as F    #提供各种函数，如激活函数、池化操作等。
from tqdm import tqdm   #显示循环进度条
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

def extract_onnx_weight(onnx_path):
    model = onnx.load(onnx_path)

    weights, names = [], []
    for t in model.graph.initializer:   #初始化器（initializers）
        weights.append(numpy_helper.to_array(t))
        names.append(t.name)
        
    onnx_weight = dict()#创建一个字典 onnx_weight，
    #将权重名称作为键，权重数据作为值，以便后续的权重查询和使用。
    
    for name, weight in zip(names, weights):
        # zip 函数将权重名称和权重值进行配对
        onnx_weight[name] = weight
    return onnx_weight

if __name__ == '__main__':

    IMG_PATH = 'dog.jpg'
    ONNX_PATH = 'deit_fp32_0812.onnx'
    OUT_TXT_PATH = "data.txt"

    w = [208,232,231,246, 264, 278, 305, 329, 328, 343, 361, 375, 402, 426, 425, 440, 458, 472, 499, 523, 522, 537, 555, 569, 596, 620, 619, 634, 652, 666, 693, 717, 716, 731, 749, 763, 790, 814, 813, 828, 846, 860, 887, 911, 910, 925, 943, 957, 984, 1008, 1007, 1022, 1040, 1054, 1081, 1105, 1104, 1119, 1137, 1151, 1178, 1202, 1201, 1216, 1234, 1248, 1275, 1299, 1298, 1313, 1331]           #手动输入onnx的w
    a = [207,227,238,245, 263, 277, 304, 324, 335, 342, 360, 374, 401, 421, 432, 439, 457, 471, 498, 518, 529, 536, 554, 568, 595, 615, 626, 633, 651, 665, 692, 712, 723, 730, 748, 762, 789, 809, 820, 827, 845, 859, 886, 906, 917, 924, 942, 956, 983, 1003, 1014, 1021, 1039, 1053, 1080, 1100, 1111, 1118, 1136, 1150, 1177, 1197, 1208, 1215, 1233, 1247, 1274, 1294, 1305, 1312, 1330]

    print(len(w))
    print(len(a))
    
    torch.manual_seed(999)
    #打印数组元素的精度设置为10位小数（precision=10）
    # 并且不限制数组的打印阈值（threshold=np.inf）
    torch.set_printoptions(precision=10,sci_mode=True)
    torch.set_printoptions(threshold=np.inf)

    # activation
    #打开图像文件，将其转换为NumPy数组，并进行一些形状的变换。
    img = Image.open(IMG_PATH).convert("RGB")
    img = np.asarray(img, np.float32)/255.0
    img= img.transpose(2,0,1).reshape(1,3,224,224)
    

    # weight
    onnx_path_list = [ONNX_PATH]
    
    out_file = OUT_TXT_PATH
    if os.path.exists(out_file):
        os.remove(out_file)
        print("文件已存在并已删除。")
    else:
        print("文件不存在。")

    final_time = np.empty((100))
    mac_cnt = np.empty((100))

    #外层循环，迭代处理权重和激活参数列表中的每一对参数
    for layerX in range(len(w)):
        start_time = time.time()
        print("This is layer "+ str(layerX))

        print(w[layerX])
        WEIGHT_NAME = str(w[layerX])        #输入w
        MED_LAYER_NAME = str(a[layerX])      #输入a

        # activate
        intermediate_layer_name = WEIGHT_NAME  
        model = onnx.load(ONNX_PATH)
        #加载ONNX模型，并设置要提取的中间层名称。
        for node in model.graph.node:
            for output in node.output:
                if(output==intermediate_layer_name):
                    model.graph.output.extend([onnx.ValueInfoProto(name=output)])
        session = onnxruntime.InferenceSession(model.SerializeToString())
        
        
        # model input
        inputs = {'x': img}
        # infer
        #使用ONNX Runtime执行模型推理。
        intermediate_layer_output = session.run([intermediate_layer_name], inputs)[0]

        weight = torch.from_numpy(intermediate_layer_output)
        ###print("##################### weight： ##################### "+ weight)
        
        
        # activate
        intermediate_layer_name = MED_LAYER_NAME  
        model = onnx.load(ONNX_PATH)
        for node in model.graph.node:
            for output in node.output:
                if(output==intermediate_layer_name):
                    model.graph.output.extend([onnx.ValueInfoProto(name=output)])
        session = onnxruntime.InferenceSession(model.SerializeToString())
        # model input
        inputs = {'x': img}
        # infer
        intermediate_layer_output = session.run([intermediate_layer_name], inputs)[0]

        act = torch.from_numpy(intermediate_layer_output)
        ###print("##################### act：##################### "+ act)
        ######################################################################
        
        #提取模型的权重和激活参数，并进行一些形状的修改，以确保它们适用于后续的操作。
        w_shape = weight.shape 
        a_shape = act.shape

        print('w_shape:',weight.shape)
        print('a_shape:',act.shape)

        if(weight.ndim == 2):
            weight = weight.reshape(1,w_shape[0],w_shape[1])
        else:
            weight = weight.reshape(w_shape[1],w_shape[2],w_shape[3])
      
        if(act.ndim == 4):
            act = act.reshape(a_shape[1],a_shape[2],a_shape[3])
            print(act.shape)
        
        print('after modify:')
        print('w_shape:',weight.shape)
        print('a_shape:',act.shape)
        w_shape = weight.shape 
        a_shape = act.shape



        a_64 = np.empty(64)
        w_64 = np.empty(64)
        cnt = 0
        layer_time = 0

        print(a_shape[2])
        print(w_shape[1])
        assert (a_shape[2] == w_shape[1]), "error !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!."
        m_cnt = 0
        
        #在嵌套的循环中，对权重和激活参数执行一些操作，
        # 包括从它们中提取数据，然后将它们传递给 bitlet_time 函数，以计算时间和MAC计数。
        for k1 in range(w_shape[0]):
            for k2 in range(a_shape[0]):
                for k3 in range(a_shape[1]):
                    print(k3,end = " ")
                    for k4 in range(int(w_shape[2]/10)):
                        for k5 in range(a_shape[2]):
                            a_64[cnt] = act[k1][k3][k5]
                            w_64[cnt] = weight[k2][k5][k4]
                            cnt = cnt + 1
                            if(cnt == 64):
                                cnt = 0
                                m_cnt = m_cnt + 1
                                layer_time = layer_time + bitlet_time(a_64,w_64)
                print("")
        #计算每一对权重和激活参数的时间和MAC计数，
        # 并将这些值存储在 final_time 和 mac_cnt 数组中。
        end_time = time.time()
        final_time[layerX] = layer_time
        mac_cnt[layerX] = m_cnt
        print(layer_time)
        print(m_cnt)
        print("cp_time",end_time - start_time)


In [ ]:
import numpy as np

total_time = 0
total_mac_cnt = 0
for i in range(71):
    total_time = total_time + final_time[i]
    total_maccnt = total_mac_cnt + mac_cnt[i]
print("total_time is ", total_time,"total_mac_cnt is ", total_maccnt)


xilinx_time_1 = total_mac_cnt*49
xilinx_time_2 = total_mac_cnt*283
print(xilinx_time_1)
print(xilinx_time_2)
print(("方案一： ",xilinx_time_1-total_time)/xilinx_time_1)
print(("方案二： ",xilinx_time_2-total_time)/xilinx_time_2)

In [ ]:
np.save("prun0-25-cp_time.npy",compute_time)
np.save("prun0-25-final_time.npy",final_time)
b = np.load("prun0-25.npy")
print(b.shape)

In [ ]:
for i in range(10):
    for j in range(0,25):
        print(b[i][j])